# Запрос №1
Напишите запрос, выбирающий информацию об имени и фамилии 
сотрудника (таблица employees), а также о названии отдела, в котором он 
работает (таблица departments). При этом выберите только сотрудников
отдела IT

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').getOrCreate()

jobDf = spark.read.parquet('data/jobs.parquet')
empDf = spark.read.parquet('data/employees.parquet')
df = empDf.join(jobDf, empDf.job_id == jobDf.job_id, "inner") \
    .filter("job_title='Sales Representative'").sort("salary", "first_name")
df.select("first_name", "last_name", "salary", "job_title").show(11, truncate=False)


+----------+---------+------+--------------------+
|first_name|last_name|salary|job_title           |
+----------+---------+------+--------------------+
|Sundita   |Kumar    |6100.0|Sales Representative|
|Amit      |Banda    |6200.0|Sales Representative|
|Charles   |Johnson  |6200.0|Sales Representative|
|Sundar    |Ande     |6400.0|Sales Representative|
|David     |Lee      |6800.0|Sales Representative|
|Kimberely |Grant    |7000.0|Sales Representative|
|Oliver    |Tuvault  |7000.0|Sales Representative|
|Sarath    |Sewall   |7000.0|Sales Representative|
|Mattea    |Marvins  |7200.0|Sales Representative|
|Elizabeth |Bates    |7300.0|Sales Representative|
|William   |Smith    |7400.0|Sales Representative|
+----------+---------+------+--------------------+
only showing top 11 rows



## Запрос №2
Напишите запрос, выбирающий информацию об имени и фамилии 
сотрудника, названии отдела, в котором он работает, и стране, в которой он 
размещён. При этом запрос должен выбирать только информацию о
сотрудниках из соединенных штатов, работающих в отделах «Shipping» и 
«Finance».

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').getOrCreate()

depDf = spark.read.parquet('data/department.parquet')
locDf = spark.read.parquet('data/locations.parquet')
cntryDf = spark.read.parquet('data/countries.parquet')
empDf = spark.read.parquet('data/employees.parquet')

df = empDf.join(depDf, empDf.department_id == depDf.department_id, "inner")
df = df.join(locDf, df.location_id == locDf.location_id, "inner")
df = df.join(cntryDf, df.country_id == cntryDf.country_id, "inner")
df.filter("country_name='United States of America'") \
    .filter((df.department_name == 'Shipping') | (df.department_name == 'Finance')) \
    .select("first_name", "last_name", "department_name", "country_name").show(11, truncate=False)


+-----------+---------+---------------+------------------------+
|first_name |last_name|department_name|country_name            |
+-----------+---------+---------------+------------------------+
|Nancy      |Greenberg|Finance        |United States of America|
|Daniel     |Faviet   |Finance        |United States of America|
|John       |Chen     |Finance        |United States of America|
|Ismael     |Sciarra  |Finance        |United States of America|
|Jose Manuel|Urman    |Finance        |United States of America|
|Luis       |Popp     |Finance        |United States of America|
|Matthew    |Weiss    |Shipping       |United States of America|
|Adam       |Fripp    |Shipping       |United States of America|
|Payam      |Kaufling |Shipping       |United States of America|
|Shanta     |Vollman  |Shipping       |United States of America|
|Kevin      |Mourgos  |Shipping       |United States of America|
+-----------+---------+---------------+------------------------+
only showing top 11 rows


## Запрос №3
Напишите запрос, выбирающий информацию по всем служащим, 
нанятым раньше своих менеджеров. При этом выведите фамилии и даты 
найма самих служащих, а также фамилии и даты найма их менеджеров. 
Пример возможного результата выполнения запроса представлен в 
таблице ниже. Поля должны называться так же, как указано в примере.

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import BooleanType

spark = SparkSession.builder.master('local').getOrCreate()

empDf = spark.read.parquet('data/employees.parquet')

cmp = udf(lambda a, b: tuple(map(int, a.split('.'))) < tuple(map(int, b.split('.'))), BooleanType())

df = empDf.withColumnRenamed("last_name", "Фамилия_Р").withColumnRenamed("hire_date", "dt_w")
df2 = empDf.select("employee_id", "last_name", "hire_date")
df = df.join(df2, df.manager_id == df2.employee_id, "inner") \
    .where(cmp(col("dt_w"), col("hire_date"))).withColumnRenamed("last_name", "Фамилия_М") \
    .withColumnRenamed("hire_date", "Дата_М").withColumnRenamed("dt_w", "Дата_Р")
df.select("Фамилия_Р", "Дата_Р", "Фамилия_М", "Дата_М").show(10, truncate=False)


+---------+----------+---------+----------+
|Фамилия_Р|Дата_Р    |Фамилия_М|Дата_М    |
+---------+----------+---------+----------+
|De Haan  |13.01.2001|King     |17.06.2003|
|Hunold   |03.01.2006|De Haan  |13.01.2001|
|Greenberg|17.08.2002|Kochhar  |21.09.2005|
|Faviet   |16.08.2002|Greenberg|17.08.2002|
|Urman    |07.03.2006|Greenberg|17.08.2002|
|Popp     |07.12.2007|Greenberg|17.08.2002|
|Raphaely |07.12.2002|King     |17.06.2003|
|Fripp    |10.04.2005|King     |17.06.2003|
|Kaufling |01.05.2003|King     |17.06.2003|
|Vollman  |10.10.2005|King     |17.06.2003|
+---------+----------+---------+----------+
only showing top 10 rows



## Запрос №4
Напишите запрос, выбирающий информацию о фамилии, имени и 
окладе всех служащих, оклад которых выше среднего. Отсортируйте 
выходные данные в порядке увеличения окладов. Пример возможного 
результата выполнения запроса представлен в таблице ниже. Поля 
должны называться так же, как указано в примере

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg

spark = SparkSession.builder.master('local').getOrCreate()

empDf = spark.read.parquet('data/employees.parquet')

df = empDf.filter(empDf.salary > empDf.select(avg('salary')).collect()[0][0]).sort("salary", 'last_name') \
    .withColumnRenamed("last_name", "Фамилия_Р").withColumnRenamed("first_name", "Имя").withColumnRenamed("salary",
                                                                                                          "Оклад")
df.select("Фамилия_Р", "Имя", "Оклад").show(10, truncate=False)


+---------+---------+------+
|Фамилия_Р|Имя      |Оклад |
+---------+---------+------+
|Mavris   |Susan    |6500.0|
|Vollman  |Shanta   |6500.0|
|Lee      |David    |6800.0|
|Popp     |Luis     |6900.0|
|Grant    |Kimberely|7000.0|
|Sewall   |Sarath   |7000.0|
|Tuvault  |Oliver   |7000.0|
|Marvins  |Mattea   |7200.0|
|Bates    |Elizabeth|7300.0|
|Smith    |William  |7400.0|
+---------+---------+------+
only showing top 10 rows



## Запрос №5
Напишите запрос, выбирающий имена и фамилии сотрудников на 
основе информации из таблицы employees. Должны быть выбраны записи 
только для тех сотрудников, которые выполняют менеджерские функции 
(для которых их номер employee_id встречается в столбце manager_id той же 
таблицы). Решите эту задачу при помощи многострочного подзапроса. 
Пример возможного результата выполнения запроса представлен в 
таблице ниже. Поля должны называться так же, как указано в примере

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession.builder.master('local').getOrCreate()

empDf = spark.read.parquet('data/employees.parquet')
managers = empDf.rdd.map(lambda x: x.manager_id).collect()
df = empDf.where(col("employee_id").isin(managers)) \
    .withColumnRenamed("first_name", "Имя").withColumnRenamed("last_name", "Фамилия")
df.sort(df.employee_id.desc()).select("Имя", "Фамилия").show()

+---------+---------+
|      Имя|  Фамилия|
+---------+---------+
|  Shelley|  Higgins|
|  Michael|Hartstein|
|    Eleni|  Zlotkey|
|   Gerald|Cambrault|
|  Alberto|Errazuriz|
|    Karen| Partners|
|     John|  Russell|
|    Kevin|  Mourgos|
|   Shanta|  Vollman|
|    Payam| Kaufling|
|     Adam|    Fripp|
|  Matthew|    Weiss|
|      Den| Raphaely|
|    Nancy|Greenberg|
|Alexander|   Hunold|
|      Lex|  De Haan|
|    Neena|  Kochhar|
|   Steven|     King|
+---------+---------+

